# previoius implementation

In [ ]:
def iou(a, b):
    """Compute the Intersection-Over-Union of two sets of boxes."""
    area_a = (a[:, 3] - a[:, 1]) * (a[:, 2] - a[:, 0])
    area_b = (b[:, 3] - b[:, 1]) * (b[:, 2] - b[:, 0])

    intersection_top_left = torch.max(a[:, None, :2], b[:, :2])
    intersection_bottom_right = torch.min(a[:, None, 2:], b[:, 2:])
    intersection_area = (intersection_bottom_right - intersection_top_left).clamp(min=0).prod(2)

    return intersection_area / (area_a[:, None] + area_b - intersection_area)



def get_precision(predictions, ground_truth, unknown=True):
    # Compute TP and FP for ensemble predictions of the unknown class
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
    tp, fp = compute_tp_fp(predictions, ground_truth)
    
    # Calculate precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    return precision
    

def get_recall(predictions, ground_truth, iou_threshold=0.1, unknown=True):
    """Compute recall for the ensemble predictions."""
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
            
    tp, fp = compute_tp_fp(predictions, ground_truth, iou_threshold)
    fn = len(ground_truth) - tp
    
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return recall


In [31]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB_backup/"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
files = [f.split("_")[0] for f in all_files if f.endswith("t1.pth.txt")]

print(len(files))


4952


In [34]:
unknown_recall = []
known_precision = []
general_recall = []
general_precision = []

for filename in files:
    filepaths = {
    "ground_truth": f"output/from_dataset/filtered_unknown/SOWODB_backup/{filename}_gt.txt",
    "model_1": f"output/from_dataset/filtered_unknown/SOWODB_backup/{filename}_t1.pth.txt",
    }
    # Storing the data from each file in a dictionary
    data = {}
    for key, filepath in filepaths.items():
        with open(filepath, "r") as file:
            data[key] = file.readlines()
    
    h, w = map(int, data["ground_truth"][-1].strip().split(": ")[1].split(", "))
    
    # Displaying the first few lines of each file to understand the data format
    sample_data = {key: value[:5] for key, value in data.items()}
    import re
    import torch
    
    def parse_bbox(tensor_string):
        """Parse tensor string to extract bounding box coordinates."""
        coords = list(map(float, re.findall(r'([-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|[-+]?\d+)', tensor_string)))
        return torch.tensor(coords)
    
    def parse_data_line(line):
        """Parse a line from the data files to extract bounding box, label, ID, and probability."""
        bbox = parse_bbox(re.search(r'tensor\((.*?)\)', line).group(0))
        label = re.search(r'LABEL: (.*?)(?: \(|$)', line).group(1)
        label_id = int(re.search(r'ID: (\d+)', line).group(1))
        prob = float(re. search(r'PROB: (.+)', line).group(1))
        return {'bbox': bbox, 'label': label, 'label_id': label_id, 'prob': prob}
    
    
    # Parsing the data again
    parsed_data = {key: [parse_data_line(line) for line in value[:-1]] for key, value in data.items()}
    unknown_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=True))
    known_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=False))
    # general_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))
    # general_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))


AttributeError: 'NoneType' object has no attribute 'group'

In [35]:
data

{'ground_truth': ['BBOX: tensor([114.5540, 729.9828, 362.4413, 990.8251]), LABEL: dog (ID: 8), PROB: 1.0\n',
  'BBOX: tensor([ 632.8638,  703.7109,  790.6103, 1000.2078]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([118.3099, 439.1156, 144.6009, 506.6718]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([330.5164, 576.1047, 362.4413, 681.1922]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([ 176.5258,  495.4125,  800.0000, 1182.2344]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([210.3286, 489.7828, 366.1972, 658.6734]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([202.8169, 395.9547, 292.9577, 534.8204]), LABEL: person (ID: 18), PROB: 1.0\n',
  'BBOX: tensor([561.5023, 482.2766, 584.0375, 499.1656]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([550.2347, 425.9797, 593.4272, 510.4250]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([582.1596, 469.1406, 629.1080, 517.9312]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  

In [29]:
def mean(lst): 
    return sum(lst) / len(lst) 
 
# Driver Code 
print(mean(unknown_recall))
print(mean(known_precision))
print(mean(general_recall))
print(mean(general_precision))

0.14416783837650218
0.5043838308074982
0.36729391147968204
0.2782444016626013


# evaluation

In [22]:
import pandas as pd
ground_truth = pd.read_csv("./csv/bbox_data_scaled_gt.csv")
prob_opt_prediction = pd.read_csv("./csv/bbox_data_scaled_opt.csv")
prob_prediction = pd.read_csv("./csv/bbox_data_scaled.csv")
custom_01_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.01.csv")
custom_03_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.03.csv")
custom_05_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.05.csv")


In [23]:
import itertools

#OWOD splits
VOC_CLASS_NAMES_COCOFIED = [
    "airplane",  "dining table", "motorcycle",
    "potted plant", "couch", "tv"
]

BASE_VOC_CLASS_NAMES = [
    "aeroplane", "diningtable", "motorbike",
    "pottedplant",  "sofa", "tvmonitor"
]
UNK_CLASS = ["unknown"]

VOC_COCO_CLASS_NAMES={}


T1_CLASS_NAMES = [
    "aeroplane","bicycle","bird","boat","bus","car",
    "cat","cow","dog","horse","motorbike","sheep","train",
    "elephant","bear","zebra","giraffe","truck","person"
]

T2_CLASS_NAMES = [
    "traffic light","fire hydrant","stop sign",
    "parking meter","bench","chair","diningtable",
    "pottedplant","backpack","umbrella","handbag",
    "tie","suitcase","microwave","oven","toaster","sink",
    "refrigerator","bed","toilet","sofa"
]

T3_CLASS_NAMES = [
    "frisbee","skis","snowboard","sports ball",
    "kite","baseball bat","baseball glove","skateboard",
    "surfboard","tennis racket","banana","apple","sandwich",
    "orange","broccoli","carrot","hot dog","pizza","donut","cake"
]

T4_CLASS_NAMES = [
    "laptop","mouse","remote","keyboard","cell phone","book",
    "clock","vase","scissors","teddy bear","hair drier","toothbrush",
    "wine glass","cup","fork","knife","spoon","bowl","tvmonitor","bottle"
]

VOC_COCO_CLASS_NAMES["OWDETR"] = tuple(itertools.chain(T1_CLASS_NAMES, T2_CLASS_NAMES, T3_CLASS_NAMES, T4_CLASS_NAMES, UNK_CLASS))


VOC_CLASS_NAMES = [
"aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
"chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
"pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

VOC_CLASS_NAMES_COCOFIED = [
    "airplane",  "dining table", "motorcycle",
    "potted plant", "couch", "tv"
]

BASE_VOC_CLASS_NAMES = [
    "aeroplane", "diningtable", "motorbike",
    "pottedplant",  "sofa", "tvmonitor"
]

T2_CLASS_NAMES = [
    "truck", "traffic light", "fire hydrant", "stop sign", "parking meter",
    "bench", "elephant", "bear", "zebra", "giraffe",
    "backpack", "umbrella", "handbag", "tie", "suitcase",
    "microwave", "oven", "toaster", "sink", "refrigerator"
]

T3_CLASS_NAMES = [
    "frisbee", "skis", "snowboard", "sports ball", "kite",
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake"
]

T4_CLASS_NAMES = [
    "bed", "toilet", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "book", "clock",
    "vase", "scissors", "teddy bear", "hair drier", "toothbrush",
    "wine glass", "cup", "fork", "knife", "spoon", "bowl"
]
VOC_COCO_CLASS_NAMES["TOWOD"] = tuple(itertools.chain(VOC_CLASS_NAMES, T2_CLASS_NAMES, T3_CLASS_NAMES, T4_CLASS_NAMES, UNK_CLASS))
VOC_COCO_CLASS_NAMES["VOC2007"] = tuple(itertools.chain(VOC_CLASS_NAMES, T2_CLASS_NAMES, T3_CLASS_NAMES, T4_CLASS_NAMES, UNK_CLASS))



In [24]:
gt_data = ground_truth#[ground_truth['label']!='unknown']
pred_data = prob_opt_prediction#[custom_01_prediction['label']!='unknown']
pred_data

,img_name,x_min,y_min,x_max,y_max,label,prob


In [21]:
from datasets.open_world_eval import voc_eval
import torch

from collections import OrderedDict, defaultdict

num_classes = len(VOC_COCO_CLASS_NAMES["OWDETR"])
AP = torch.zeros(num_classes, 1)

all_recs = defaultdict(list)
all_precs = defaultdict(list)
recs = defaultdict(list)
precs = defaultdict(list)
num_unks = defaultdict(list)
unk_det_as_knowns = defaultdict(list)
tp_plus_fp_cs = defaultdict(list)
fp_os = defaultdict(list)

ovthresh = 50

for class_label_ind, class_label in enumerate(VOC_COCO_CLASS_NAMES["OWDETR"]):
    detpath = [
        f'{str(row.img_name).zfill(12)} {row.prob} {row.x_min} {row.y_min} {row.x_max} {row.y_max}\n'
        for index, row in pred_data.iterrows() if row.label == class_label
    ]
    annopath = [f'/training_data_2/yuetian/OWOD/Annotations/{str(row.img_name).zfill(12)}.xml' for index, row in pred_data.iterrows()]
    imagesetfile = [f'{str(row.img_name).zfill(12)}' for index, row in pred_data.iterrows()]
    
    ovthresh_ind, _ = map(list(range(50, 100, 5)).index, [50, 75])

    print(class_label + " has " + str(len(detpath)) + " predictions.")

    
    rec, prec, AP[class_label_ind, ovthresh_ind], unk_det_as_known, num_unk, tp_plus_fp_closed_set,fp_open_set = (voc_eval(detpath,
             annopath,
             imagesetfile,
             class_label,
             ovthresh=0.5,
             use_07_metric=False,
             known_classes=T1_CLASS_NAMES))
    
    AP[class_label_ind, ovthresh_ind] = AP[class_label_ind, ovthresh_ind] * 100
    all_precs[ovthresh].append(prec)
    num_unks[ovthresh].append(num_unk)
    unk_det_as_knowns[ovthresh].append(unk_det_as_known)
    tp_plus_fp_cs[ovthresh].append(tp_plus_fp_closed_set)
    fp_os[ovthresh].append(fp_open_set)
    try:
        recs[ovthresh].append(rec[-1] * 100)
        precs[ovthresh].append(prec[-1] * 100)
    except:
        recs[ovthresh].append(0.)
        precs[ovthresh].append(0.)
    

aeroplane has 4464 predictions.
bicycle has 1476 predictions.
bird has 1416 predictions.
boat has 2004 predictions.
bus has 6178 predictions.
car has 1490 predictions.
cat has 7236 predictions.
cow has 4168 predictions.
dog has 8804 predictions.
horse has 1256 predictions.
motorbike has 3234 predictions.
sheep has 2660 predictions.
train has 1514 predictions.
elephant has 978 predictions.
bear has 83532 predictions.
zebra has 3140 predictions.
giraffe has 912 predictions.
truck has 2949 predictions.
person has 588 predictions.
traffic light has 2652 predictions.


/home/cheny63/experiment/PROB/datasets/open_world_eval.py:436: RuntimeWarning: invalid value encountered in divide
  rec = tp / float(npos)


fire hydrant has 0 predictions.
stop sign has 0 predictions.
parking meter has 0 predictions.
bench has 0 predictions.
chair has 0 predictions.


KeyboardInterrupt: 

In [12]:
o50, _ = map(list(range(50, 100, 5)).index, [50, 75])
mAP = float(AP.mean())
mAP50 = float(AP[:, o50].mean())
fmt='{:.06f}'
print('detection mAP50:', fmt.format(mAP50))
print('detection mAP:', fmt.format(mAP))

detection mAP50: 0.007192
detection mAP: 0.007192


In [13]:
print("Known AP50: " + str(AP[:, o50][:20].mean()))

Known AP50: tensor(0.0288)


In [7]:
print("Unknown Recall50: " + str(recs[50][-1]))

Unknown Recall50: 0.23175344915713447


In [132]:
mpre

array([1., 1., 0.])

In [11]:
AP[19] = torch.tensor([0])